In [1]:
pwd

'C:\\'

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("EizeShem").getOrCreate()

In [4]:
df = spark.read.json('stam.json')

In [11]:
df

DataFrame[age: bigint, name: string]

In [6]:
df.show()

+----+-----+
| age| name|
+----+-----+
|null| Shay|
|  25|Boris|
|  29| Omri|
+----+-----+



In [7]:
df.columns

['age', 'name']

In [9]:
df.describe().show()

+-------+------------------+-----+
|summary|               age| name|
+-------+------------------+-----+
|  count|                 2|    3|
|   mean|              27.0| null|
| stddev|2.8284271247461903| null|
|    min|                25|Boris|
|    max|                29| Shay|
+-------+------------------+-----+



In [10]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

In [12]:
data_schema = [StructField("age", IntegerType(), True),StructField("name", StringType(), True)]

In [13]:
final_struct = StructType(fields = data_schema)

In [14]:
df = spark.read.json('stam.json', schema=final_struct)

In [16]:
df.show()

+----+-----+
| age| name|
+----+-----+
|null| Shay|
|  25|Boris|
|  29| Omri|
+----+-----+



In [17]:
df['age']

Column<b'age'>

In [18]:
type(df['age'])

pyspark.sql.column.Column

In [21]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  25|
|  29|
+----+



In [22]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [24]:
df.head(2)

[Row(age=None, name='Shay'), Row(age=25, name='Boris')]

In [26]:
df.withColumn('newage',df['age']*2).show()

+----+-----+------+
| age| name|newage|
+----+-----+------+
|null| Shay|  null|
|  25|Boris|    50|
|  29| Omri|    58|
+----+-----+------+



In [27]:
df.createOrReplaceTempView("stamtavla")

In [31]:
sql_results = spark.sql("select * from stamtavla where age<27")

In [32]:
sql_results.show()

+---+-----+
|age| name|
+---+-----+
| 25|Boris|
+---+-----+



## APPLE STOCK ANALYSIS


In [34]:
df = spark.read.csv("appl_stock.csv",inferSchema=True,header=True)

In [36]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [37]:
df.filter("Close<500").show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [39]:
df.filter("Close<500").select(['Open','Close']).show()

+------------------+------------------+
|              Open|             Close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



In [42]:
df.filter((df["Close"]<200) & (df["Open"]>200)).show()

+-------------------+------------------+----------+----------+----------+---------+------------------+
|               Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+-------------------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22 00:00:00|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28 00:00:00|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29 00:00:00|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+-------------------+------------------+----------+----------+----------+---------+------------------+



## Group By in Spark

In [43]:
df = spark.read.csv('sales_info.csv',inferSchema=True,header=True)

In [44]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [45]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [46]:
df.groupBy("Company")

In [48]:
df.groupBy("Company").mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [49]:
df.groupBy("Company").count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [50]:
df.groupBy("Company").sum().show()

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   APPL|    1480.0|
|   GOOG|     660.0|
|     FB|    1220.0|
|   MSFT|     967.0|
+-------+----------+



In [51]:
from pyspark.sql.functions import countDistinct,avg,stddev

In [52]:
df.select(countDistinct("Sales")).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [53]:
df.select(countDistinct("Sales").alias("Distinct Sales")).show()

+--------------+
|Distinct Sales|
+--------------+
|            11|
+--------------+



In [54]:
df.select(avg('Sales')).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [55]:
df.select(stddev('Sales')).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+

